# Submission Machine Learning Pipeline

Nama : M Mahfudl Awaludin

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [2]:
PIPELINE_NAME = "mahfudl-pipeline"
SCHEMA_PIPELINE_NAME = "happy-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Pengolahan Data

Pembuatan komponen **ExampleGen**

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 7
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
from pprint import PrettyPrinter
pp = PrettyPrinter()

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

tfrecord_train_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_train_filenames, compression_type='GZIP')

for tfrecord in dataset.take(2):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "events"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "happy"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "housecost"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "infoavail"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "policetrust"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "schoolquality"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "streetquality"
    value {
      int64_list {
        value: 2
      }
    }
  }
}

features {
  feature {
    key: "events"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "happy"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "housecost"
    value {
      int64_list {
        value: 2
      }

Pembuatan komponen **StatisticGen**

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

Pembuatan komponen **SchemaGen**

In [9]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'events',INT,required,,-
'happy',INT,required,,-
'housecost',INT,required,,-
'infoavail',INT,required,,-
'policetrust',INT,required,,-
'schoolquality',INT,required,,-
'streetquality',INT,required,,-


Pembuatan komponen **ExampleValidator**

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 10
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

Pembuatan komponen **Transform**

In [13]:
TRANSFORM_MODULE_FILE = r"C:\Users\user\Music\submission-mlops\happy_transform.py"

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
LABEL_KEY = "happy"
FEATURE_KEYS = [
    "infoavail",
    "housecost",
    "schoolquality",
    "policetrust",
    "streetquality",
    "events"
]
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
   # Loop through the feature columns and apply transformations (e.g., scaling or casting)
    for feature_key in FEATURE_KEYS:
        outputs[transformed_name(feature_key)] = tf.cast(inputs[feature_key], tf.float32)
    
    # Transforming the 'FetalHealth' label into integer type if it's not already
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Overwriting C:\Users\user\Music\submission-mlops\happy_transform.py


In [15]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\mahfudl-pipeline\Transform\transform_graph\11\.temp_path\tftransform_tmp\007c1f32e2304009bef9560ecacffa4c\assets


INFO:tensorflow:Assets written to: pipelines\mahfudl-pipeline\Transform\transform_graph\11\.temp_path\tftransform_tmp\007c1f32e2304009bef9560ecacffa4c\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 11
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

### Pengembangan & Validasi Model

In [20]:
TRAINER_MODULE_FILE = "happy_trainer.py"

In [33]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import os  
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "happy"  # Updated to match the dataset
FEATURE_KEYS = [
    "infoavail",
    "housecost",
    "schoolquality",
    "policetrust",
    "streetquality",
    "events"
]

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64) -> tf.data.Dataset:
    """Get post_transform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    return dataset

# Model parameters
embedding_dim = 16

def model_builder():
    """Build machine learning model with improvements"""
    inputs = {transformed_name(key): tf.keras.Input(shape=(1,), name=transformed_name(key), dtype=tf.float32) for key in FEATURE_KEYS}
    
    # Concatenate all features
    concatenated_inputs = layers.Concatenate()(list(inputs.values()))
    
    # First dense layer with more units
    x = layers.Dense(256, activation='relu')(concatenated_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)  # Reduce dropout to 0.3
    
    # Second dense layer with more units
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)  # Reduce dropout to 0.3
    
    # Third dense layer with more units
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Output layer
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.001),  # Slightly increase learning rate
        metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    
    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=20)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    
    # Build the model
    model = model_builder()
    
    # Train the model
    model.fit(x=train_set,
              validation_data=val_set,
              callbacks=[tensorboard_callback, es, mc],
              steps_per_epoch=1000, 
              validation_steps=1000,
              epochs=20)  # Increased the number of epochs
    
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting happy_trainer.py


In [34]:
from tfx.proto import trainer_pb2

trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

In [35]:
interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 infoavail_xf (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 housecost_xf (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 schoolquality_xf (InputLayer)  [(None, 1)]          0           []                               
                                                                                                  
 policetrust_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                            


Epoch 1: val_binary_accuracy improved from -inf to 0.63636, saving model to pipelines\mahfudl-pipeline\Trainer\model\15\Format-Serving
INFO:tensorflow:Assets written to: pipelines\mahfudl-pipeline\Trainer\model\15\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\mahfudl-pipeline\Trainer\model\15\Format-Serving\assets


1000/1000 [==============================] - 7s 4ms/step - loss: 0.7003 - binary_accuracy: 0.6100 - precision: 0.6229 - recall: 0.5929 - val_loss: 0.6348 - val_binary_accuracy: 0.6364 - val_precision: 0.6364 - val_recall: 1.0000
INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\mahfudl-pipeline\Trainer\model\15\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\mahfudl-pipeline\Trainer\model\15\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 15
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

Membuat komponen **Resolver**

In [39]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 

model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 16
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

Membuat komponen **Evaluator**

In [40]:
import tensorflow_model_analysis as tfma 

# Update the label key to match the dataset
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='happy')],  # Changed from 'is_sarcastic' to 'happy'
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': 0.0001})
                )
            )
        ])
    ]
)

from tfx.components import Evaluator

# Ensure that the example_gen and trainer outputs are correctly referenced
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)

In [41]:
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 17
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [42]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [43]:
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [44]:
# Print validation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(eval_result))

validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 1
  }
}



Membuat komponen Pusher

In [45]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 

pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/happy-detection-model'))

)

In [46]:
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 18
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))